In [ ]:
from google.colab import files
files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!kaggle datasets download -d hungryfool/av-recommendation-systems
!unzip /content/av-recommendation-systems.zip

Saving kaggle.json to kaggle.json
 57% 5.00M/8.84M [00:00<00:00, 46.6MB/s]
100% 8.84M/8.84M [00:00<00:00, 56.7MB/s]
Archive:  /content/av-recommendation-systems.zip
  inflating: sample_submission_J0OjXLi_DDt3uQN.csv  
  inflating: test_HLxMpl7/test.csv   
  inflating: train_mddNHeX/challenge_data.csv  
  inflating: train_mddNHeX/train.csv  


## Imports

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,Input,BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
import time
%matplotlib inline
# Classification
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
import lightgbm as lgb
#import catboost as cat
# from catboost import CatBoostRegressor
# import sweetviz

# Preprocessing
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from mlxtend.classifier import StackingClassifier
from datetime import datetime, timedelta
from sklearn.naive_bayes import MultinomialNB

from sklearn import preprocessing
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import SGDClassifier
# !pip install pycaret
# from pycaret.classification import *from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn.linear_model import *
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.tokenize import TreebankWordTokenizer
import matplotlib.pyplot as plt
%matplotlib inline
import re


import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


## Data

In [ ]:
test = pd.read_csv('/content/test_HLxMpl7/test.csv')
train = pd.read_csv('/content/train_mddNHeX/train.csv')
challenges = pd.read_csv('/content/train_mddNHeX/challenge_data.csv')
sampsub = pd.read_csv('/content/sample_submission_J0OjXLi_DDt3uQN.csv')

In [ ]:
label = train[train.challenge_sequence > 10][['user_id','challenge']]
label.rename(columns={'challenge':'label'},inplace=True)

In [ ]:
label.head(10)

,user_id,label
10,4576,CI24958
11,4576,CI23667
12,4576,CI23691
23,4580,CI24915
24,4580,CI25727
25,4580,CI26051
36,4581,CI26163
37,4581,CI26166
38,4581,CI26167
49,4582,CI25135


In [ ]:
df = train[train.challenge_sequence <= 10].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()

In [ ]:
df = df.merge(label)

In [ ]:
df.head()

,user_id,challenge,label
0,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI24958
1,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23667
2,4576,CI23714 CI23855 CI24917 CI23663 CI23933 CI2513...,CI23691
3,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI24915
4,4580,CI23663 CI23855 CI23933 CI23975 CI24530 CI2371...,CI25727


In [ ]:
df1 = df.challenge.str.split(expand=True)
df1[['user_id','label']] = df[['user_id','label']]

In [ ]:
df=df1
df.drop(columns = ['user_id'], inplace = True)
df.head()

,0,1,2,3,4,5,6,7,8,9,label
0,CI23714,CI23855,CI24917,CI23663,CI23933,CI25135,CI23975,CI25126,CI24915,CI24957,CI24958
1,CI23714,CI23855,CI24917,CI23663,CI23933,CI25135,CI23975,CI25126,CI24915,CI24957,CI23667
2,CI23714,CI23855,CI24917,CI23663,CI23933,CI25135,CI23975,CI25126,CI24915,CI24957,CI23691
3,CI23663,CI23855,CI23933,CI23975,CI24530,CI23714,CI23648,CI23781,CI23667,CI25135,CI24915
4,CI23663,CI23855,CI23933,CI23975,CI24530,CI23714,CI23648,CI23781,CI23667,CI25135,CI25727


In [ ]:
encoder = LabelEncoder()
encoder.fit(challenges['challenge_ID'])
df= df.apply(encoder.transform)
df.head()

,0,1,2,3,4,5,6,7,8,9,label
0,236,377,1439,185,455,1657,497,1648,1437,1479,1480
1,236,377,1439,185,455,1657,497,1648,1437,1479,189
2,236,377,1439,185,455,1657,497,1648,1437,1479,213
3,185,377,455,497,1052,236,170,303,189,1657,1437
4,185,377,455,497,1052,236,170,303,189,1657,2249


In [ ]:
max(df['label'])

5605

In [ ]:
df_train, df_validation = train_test_split(df.sample(frac=1,random_state=1996), test_size=0.05, random_state=123)

In [ ]:
df_train.shape, df_validation.shape

((198166, 11), (10430, 11))

In [ ]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,label
103011,4181,4187,4202,4204,4203,4186,4182,4183,2732,2691,2742
127290,1875,1918,2120,2118,2124,1381,2122,2121,1194,1802,624
178438,1452,1460,1464,1454,1451,1449,1575,1456,1448,1702,1699
184566,497,521,1648,185,1647,1782,4118,4106,1546,438,2572
207719,1828,2284,2663,1481,2582,1052,455,2563,951,898,2578


In [ ]:
# encoder = LabelEncoder()
# encoder.fit(challenges['challenge_ID'])
# df_train['brand_id_encoded'] = encoder.transform(df_train.label)
# df_validation['brand_id_encoded'] = encoder.transform(df_validation.label)
    

In [ ]:
challenges

,challenge_ID,programming_language,challenge_series_ID,total_submissions,publish_date,author_ID,author_gender,author_org_ID,category_id
0,CI23478,2,SI2445,37.0,06-05-2006,AI563576,M,AOI100001,NaN
1,CI23479,2,SI2435,48.0,17-10-2002,AI563577,M,AOI100002,32.0
2,CI23480,1,SI2435,15.0,16-10-2002,AI563578,M,AOI100003,NaN
3,CI23481,1,SI2710,236.0,19-09-2003,AI563579,M,AOI100004,70.0
4,CI23482,2,SI2440,137.0,21-03-2002,AI563580,M,AOI100005,NaN
...,...,...,...,...,...,...,...,...,...
5601,CI29079,1,SI2864,NaN,17-06-2010,AI567059,M,AOI101717,29.0
5602,CI29080,1,SI2865,NaN,25-06-2010,AI567060,F,AOI101718,29.0
5603,CI29081,1,SI2865,NaN,25-06-2010,AI566257,M,AOI100108,29.0
5604,CI29082,1,SI2865,NaN,25-06-2010,AI563777,M,AOI100108,29.0


In [ ]:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(df_train['challenge'])

In [ ]:
# tokenizer.word_index['ci23975']

In [ ]:
# len(tokenizer.word_index)

In [ ]:
challenges.shape[0]

5606

In [ ]:
NB_WORDS = challenges.shape[0]
MAX_SEQUENCE_LENGTH = 10
N_CATEGORIES = challenges.shape[0]

In [ ]:
# sequences_train = tokenizer.texts_to_sequences(df_train['challenge'])
# sequences_validation = tokenizer.texts_to_sequences(df_validation['challenge'])

In [ ]:
x_train = df_train.drop('label',axis=1)
x_validation = df_validation.drop('label',axis=1)
x_train.reset_index(drop=True, inplace=True)
x_validation.reset_index(drop=True, inplace=True)

In [ ]:
x_train.shape

(198166, 10)

In [ ]:
y_train = df_train['label'].values
y_validation= df_validation['label'].values

In [ ]:
len(y_train)

198166

## KERAS

In [ ]:
def get_model(path='',lr=0.001):
    adam = Adam(lr=lr)
    inp = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    x = Embedding(NB_WORDS,256)(inp)
    x = BatchNormalization()(x)
    x = Bidirectional(LSTM(128, dropout=0.1, recurrent_dropout=0.1))(x)
    x = Dropout(0.4)(x)
    x = Dense(N_CATEGORIES, activation="softmax")(x)
    model = Model(inputs=inp, outputs=x)
    if path != '':
        model.load_weights(path)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [ ]:
model = get_model()

# Model callbacks
path = 'best_model_weights'
#es_callback = EarlyStopping(monitor="val_loss", patience=5)
mc_callback = ModelCheckpoint('{}.hdf5'.format(path), monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto', period=1)
callbacks = [mc_callback] #es_callback,

In [ ]:
model.fit(x_train,
          y_train,
          epochs=20,
          batch_size=1024,
          validation_data=(x_validation, y_validation),
          callbacks = callbacks
          )

# Load best weights
model = get_model('{}.hdf5'.format(path))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 198166 samples, validate on 10430 samples
Epoch 1/20
198166/198166 [==============================] - 20s 101us/step - loss: 3.7465 - accuracy: 0.1433 - val_loss: 4.3943 - val_accuracy: 0.0812
Epoch 2/20
198166/198166 [==============================] - 19s 96us/step - loss: 3.6749 - accuracy: 0.1504 - val_loss: 4.3979 - val_accuracy: 0.0822
Epoch 3/20
198166/198166 [==============================] - 19s 95us/step - loss: 3.6202 - accuracy: 0.1541 - val_loss: 4.3989 - val_accuracy: 0.0804
Epoch 4/20
198166/198166 [==============================] - 19s 96us/step - loss: 3.5753 - accuracy: 0.1565 - val_loss: 4.4121 - val_accuracy: 0.0828
Epoch 5/20
198166/198166 [==============================] - 19s 96us/step - loss: 3.5395 - accuracy: 0.1592 - val_loss: 4.4213 - val_accuracy: 0.0783
Epoch 6/20
198166/198166 [==============================] - 19s 95us/step - loss: 3.5043 - accuracy: 0.1613 - val_loss: 4.4287 - val_accuracy: 0.0753
Epoch 7/20
198166/198166 [======================

In [ ]:
df_test = test[test.challenge_sequence <= 10].groupby('user_id').challenge.aggregate(lambda x: ' '.join(x)).reset_index()
df2 = df_test.challenge.str.split(expand=True)
df2= df2.apply(encoder.transform)
x_test = df2

In [ ]:
x_test.head()

,0,1,2,3,4,5,6,7,8,9
0,377,455,1439,1437,236,185,1480,1657,2249,1052
1,185,377,1439,455,497,236,1657,1437,1480,303
2,3461,3462,3463,3464,3465,3466,3467,3469,3470,3476
3,185,377,497,236,370,455,1657,303,1052,189
4,377,497,1657,370,236,1439,451,2255,1648,435


In [ ]:
pred = model.predict(x_test,batch_size=2048)

In [ ]:
pred.shape

(39732, 5606)

In [ ]:
pred = pred.argsort(axis=1)[:,-3:][:,::-1]

In [ ]:
pred

array([[2574, 2573,  370],
       [ 370, 1648,  189],
       [3471, 3475, 3477],
       ...,
       [1480, 1883, 2773],
       [1648, 1661, 1665],
       [ 370, 1398,  709]])

In [ ]:
df_list = []
for i in range(3):
    test_11 = df_test[['user_id']]
    test_11['user_sequence'] = test_11.user_id.astype(str) + '_'+str(i+11)
    test_11['challenge'] = encoder.inverse_transform(pred[:,i])


    df_list.append(test_11[['user_sequence','challenge']])
    
pd.concat(df_list).to_csv('base_submission_4.csv',index=False)
pd.concat(df_list)

,user_sequence,challenge
0,4577_11,CI26052
1,4578_11,CI23848
2,4579_11,CI26949
3,4583_11,CI24527
4,4584_11,CI23663
...,...,...
39727,113829_13,CI24493
39728,113830_13,CI23688
39729,113831_13,CI26251
39730,113834_13,CI25143


In [ ]:
pd.concat(df_list)

,user_sequence,challenge
0,4577_11,CI26052
1,4578_11,CI23667
2,4579_11,CI26955
3,4583_11,CI24958
4,4584_11,CI24867
...,...,...
39727,113829_13,CI24418
39728,113830_13,CI24111
39729,113831_13,CI24931
39730,113834_13,CI25052
